In [2]:
import os
import getpass

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

os.environ['OPENAI_API_KEY'] = "API KEY HERE"
os.environ['ACTIVELOOP_TOKEN'] = "API KEY HERE"

In [3]:
embeddings = OpenAIEmbeddings(disallowed_special=())


In [4]:
!git clone https://github.com/Significant-Gravitas/Auto-GPT # replace any repository of your choice 

Cloning into 'Auto-GPT'...
remote: Enumerating objects: 10997, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (202/202), done.
remote: Total 10997 (delta 102), reused 169 (delta 54), pack-reused 10739
Receiving objects: 100% (10997/10997), 6.61 MiB | 12.89 MiB/s, done.
Resolving deltas: 100% (7342/7342), done.


In [5]:
import os
from langchain.document_loaders import TextLoader

root_dir = './Auto-GPT'
docs = []
for dirpath, dirnames, filenames in os.walk(root_dir):
    for file in filenames:
        try: 
            loader = TextLoader(os.path.join(dirpath, file), encoding='utf-8')
            docs.extend(loader.load_and_split())
        except Exception as e: 
            pass

In [6]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(docs)

Created a chunk of size 2329, which is longer than the specified 1000
Created a chunk of size 1827, which is longer than the specified 1000
Created a chunk of size 1607, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1067, which is longer than the specified 1000
Created a chunk of size 2480, which is longer than the specified 1000
Created a chunk of size 2043, which is longer than the specified 1000
Created a chunk of size 1490, which is longer than the specified 1000
Created a chunk of size 2277, which is longer than the specified 1000
Created a chunk of size 1055, which is longer than the specified 1000
Created a chunk of size 1030, which is longer than the specified 1000
Created a chunk of size 1680, which is longer than the specified 1000
Created a chunk of size 1080, which is longer than the specified 1000
Created a chunk of size 3054, which is longer than the specified 1000
Created a chunk of s

In [10]:
username = "rihp" # replace with your username from app.activeloop.ai
db = DeepLake(dataset_path=f"hub://{username}/auto-gpt", embedding_function=embeddings, public=True) #dataset would be publicly available
db.add_documents(texts)

Your Deep Lake dataset has been successfully created!


-

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/rihp/auto-gpt


hub://rihp/auto-gpt loaded successfully.


Evaluating ingest: 100%|██████████| 1/1 [00:31<00:00
|

Dataset(path='hub://rihp/auto-gpt', tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (920, 1536)  float32   None   
    ids      text     (920, 1)      str     None   
 metadata    json     (920, 1)      str     None   
   text      text     (920, 1)      str     None   


['9cb6b106-f1dc-11ed-804a-acde48001122',
 '9cb6b1d8-f1dc-11ed-804a-acde48001122',
 '9cb6b200-f1dc-11ed-804a-acde48001122',
 '9cb6b21e-f1dc-11ed-804a-acde48001122',
 '9cb6b246-f1dc-11ed-804a-acde48001122',
 '9cb6b264-f1dc-11ed-804a-acde48001122',
 '9cb6b282-f1dc-11ed-804a-acde48001122',
 '9cb6b2b4-f1dc-11ed-804a-acde48001122',
 '9cb6b2d2-f1dc-11ed-804a-acde48001122',
 '9cb6b2f0-f1dc-11ed-804a-acde48001122',
 '9cb6b30e-f1dc-11ed-804a-acde48001122',
 '9cb6b32c-f1dc-11ed-804a-acde48001122',
 '9cb6b340-f1dc-11ed-804a-acde48001122',
 '9cb6b35e-f1dc-11ed-804a-acde48001122',
 '9cb6b37c-f1dc-11ed-804a-acde48001122',
 '9cb6b390-f1dc-11ed-804a-acde48001122',
 '9cb6b3ae-f1dc-11ed-804a-acde48001122',
 '9cb6b3c2-f1dc-11ed-804a-acde48001122',
 '9cb6b3e0-f1dc-11ed-804a-acde48001122',
 '9cb6b3fe-f1dc-11ed-804a-acde48001122',
 '9cb6b41c-f1dc-11ed-804a-acde48001122',
 '9cb6b430-f1dc-11ed-804a-acde48001122',
 '9cb6b44e-f1dc-11ed-804a-acde48001122',
 '9cb6b462-f1dc-11ed-804a-acde48001122',
 '9cb6b48a-f1dc-

In [11]:
db = DeepLake(dataset_path="hub://rihp/auto-gpt", read_only=True, embedding_function=embeddings)

/

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/rihp/auto-gpt



/

hub://rihp/auto-gpt loaded successfully.

Deep Lake Dataset in hub://rihp/auto-gpt already exists, loading from the storage
Dataset(path='hub://rihp/auto-gpt', read_only=True, tensors=['embedding', 'ids', 'metadata', 'text'])

  tensor     htype      shape      dtype  compression
  -------   -------    -------    -------  ------- 
 embedding  generic  (920, 1536)  float32   None   
    ids      text     (920, 1)      str     None   
 metadata    json     (920, 1)      str     None   
   text      text     (920, 1)      str     None   


In [12]:
retriever = db.as_retriever()
retriever.search_kwargs['distance_metric'] = 'cos'
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
retriever.search_kwargs['k'] = 10

In [13]:
def filter(x):
    # filter based on source code
    if 'com.google' in x['text'].data()['value']:
        return False
    
    # filter based on path e.g. extension
    metadata =  x['metadata'].data()['value']
    return 'scala' in metadata['source'] or 'py' in metadata['source']

### turn on below for custom filtering
# retriever.search_kwargs['filter'] = filter

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model_name='gpt-3.5-turbo') # switch to 'gpt-4'
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

In [23]:
questions = [
    "What are the agent modules doing and how can they be improved?",
    "How can we improve the tests?",
    "How can we make more difficult challenges for the bot?" ,  
] 
chat_history = []

for question in questions:  
    result = qa({"question": question, "chat_history": chat_history})
    chat_history.append((question, result['answer']))
    print(f"-> **Question**: {question} \n")
    print(f"**Answer**: {result['answer']} \n")

-> **Question**: What are the agent modules doing and how can they be improved? 

**Answer**: I'm sorry, I don't have enough information to answer your question. Can you please provide more context or specify which agent modules you are referring to? 

-> **Question**: How can we improve the tests? 

**Answer**: Certainly! The context provided was related to testing for the Auto-GPT system. Specifically, it mentions a test called "goal_oriented" which is a part of the integration testing suite. The suggestion given is to browse the repository for issues that can be fixed and to run the aforementioned command to determine if the failure is due to a change in the prompt generation or if the Auto-GPT system itself is broken. Does this help? 

-> **Question**: How can we make more difficult challenges for the bot? 

**Answer**: To make more difficult challenges for the bot, you could consider:

1. Increasing the complexity of the tasks: Make the tasks more complex by adding more steps, mor